<a href="https://colab.research.google.com/github/shunrei9841-sudo/Guadalupe/blob/main/Parcial%202-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.stats import norm, expon
import time

# --- Parámetros de la simulación ---
N = 1000000  # Número de simulaciones
threshold = 3.0 # Umbral
mu_X = 0.0
sigma_X = 1.0

# --- Valor Analítico de Referencia ---
# P(X > 3) = 1 - CDF(3)
P_analytic = 1 - norm.cdf(threshold, loc=mu_X, scale=sigma_X)
print(f"Valor Analítico P(X > 3): {P_analytic:.9f}")
print("-" * 50)


# --- A) Simulación Simple (Monte Carlo Directo) ---
print("--- A) Simulación Simple ---")

# 1. Generar N muestras de X ~ N(0, 1)
X_simple = np.random.normal(mu_X, sigma_X, N)

# 2. Estimar la probabilidad P(X > 3)
# Función indicadora g(X) = I(X > 3)
indicator_simple = (X_simple > threshold).astype(int)

# Estimador: P_hat = E[g(X)]
P_hat_simple = np.mean(indicator_simple)

# 3. Calcular la varianza del estimador
# Var[P_hat] = Var[I(X > 3)] / N = P(1-P) / N
# Usamos la varianza muestral de la indicadora:
Var_indicator_simple = np.var(indicator_simple, ddof=1)
Var_P_hat_simple = Var_indicator_simple / N

print(f"Estimación P(X > 3) (Simple): {P_hat_simple:.9f}")
print(f"Número de éxitos (X > 3): {np.sum(indicator_simple)}")
print(f"Varianza del Estimador (Simple): {Var_P_hat_simple:.12f}")
print("-" * 50)


# --- B) Muestreo de Importancia (Importance Sampling) ---
print("--- B) Muestreo de Importancia ---")

# Usaremos una distribución exponencial como función de muestreo q(x).
# q(x) = lambda * exp(-lambda * (x - 3)) para x >= 3
# La función de muestreo debe estar concentrada en la región de interés (X > 3).
# Elegimos una exponencial desplazada: Y = 3 + Z, donde Z ~ Exp(lambda).
# Por lo tanto, E[Y] = 3 + 1/lambda.
# La PDF de Y es q(x) = f_Z(x-3) = lambda * exp(-lambda * (x - 3)) para x >= 3.

# 1. Encontrar el mejor parámetro lambda*
# La varianza del estimador de muestreo de importancia es V[L(X)] / N, donde
# L(X) = g(X) * f(X) / q(X). Para minimizar V[L(X)], q(x) debe ser proporcional a
# g(x) * f(x). En nuestro caso, g(x) = I(x > 3).
# Queremos que q(x) ~ f(x) para x > 3.

# Para la normal estándar N(0, 1), f(x) es aproximadamente proporcional a exp(-x * mu_X)
# para x grande (aproximación de cola).
# Para N(0, 1), el mejor parámetro teórico es: lambda* ≈ 3.4
# (La mejor q(x) es la exponencial con lambda* = -mu_normal + sqrt(mu_normal^2 + 2*sigma^2/x_c)
# Usando la aproximación de la cola: lambda* ≈ umbral/sigma^2 = 3/1^2 = 3.0.
# Sin embargo, la mejor $\lambda$ que minimiza la varianza de la muestra suele ser ligeramente superior.
lambda_optimal = 3.0 # Usaremos la aproximación teórica lambda* ≈ threshold/sigma^2

# Generamos N muestras de la Exponencial desplazada Y ~ 3 + Exp(lambda)
# Z = -1/lambda * ln(1 - U) donde U ~ Unif(0, 1)
U = np.random.uniform(0, 1, N)
Y_imp = threshold + (-1/lambda_optimal * np.log(1 - U))

# 2. Calcular el peso de importancia (Likelihood Ratio) L(Y)
# L(y) = f(y) / q(y)
# f(y) = normal.pdf(y)
# q(y) = expon.pdf(y - threshold, scale=1/lambda_optimal)

f_Y = norm.pdf(Y_imp, loc=mu_X, scale=sigma_X)
q_Y = expon.pdf(Y_imp - threshold, scale=1/lambda_optimal) # scale = 1/lambda

# El estimador L(Y)
L_Y = f_Y / q_Y

# Función indicadora g(Y) = I(Y > 3). Como Y siempre es >= 3, g(Y) = 1.
# Por lo tanto, el estimador es P_hat_imp = mean(L(Y))

P_hat_imp = np.mean(L_Y)

# 3. Calcular la varianza del estimador de Muestreo de Importancia
# Var[P_hat_imp] = Var[L(Y)] / N
Var_L_Y = np.var(L_Y, ddof=1)
Var_P_hat_imp = Var_L_Y / N

# 4. Porcentaje de Reducción de Varianza
Reduccion_imp = (Var_P_hat_simple - Var_P_hat_imp) / Var_P_hat_simple * 100

print(f"Parámetro lambda* utilizado: {lambda_optimal:.2f}")
print(f"Estimación P(X > 3) (Importancia): {P_hat_imp:.9f}")
print(f"Varianza del Estimador (Importancia): {Var_P_hat_imp:.12f}")
print(f"Reducción de Varianza: {Reduccion_imp:.2f}%")
print("-" * 50)

Valor Analítico P(X > 3): 0.001349898
--------------------------------------------------
--- A) Simulación Simple ---
Estimación P(X > 3) (Simple): 0.001285000
Número de éxitos (X > 3): 1285
Varianza del Estimador (Simple): 0.000000001283
--------------------------------------------------
--- B) Muestreo de Importancia ---
Parámetro lambda* utilizado: 3.00
Estimación P(X > 3) (Importancia): 0.001350083
Varianza del Estimador (Importancia): 0.000000000000
Reducción de Varianza: 100.00%
--------------------------------------------------
